# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

프레임워크는, 소프트웨어를 짤 수 있는 하나의 structure이다. 예시로는 Django, Flask, Web2Py등이 있다.(무슨 게임)


반면 라이브러리는, 프레임워크에서 코드를 짤 때 활용할 수 있는 대상이다. numpy, pandas와 같이, 이미 누군가가 특정 기능을 수행할 수 있는 함수 등을 모아놓은 것들이다.(게임내에서 어떠한 스킬) 

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 수집된 데이터를 바탕으로 컴퓨터가 스스로 학습할 수 있게 하는 방법이다. 알고리즘을 바탕으로 데이터를 분석, 학습하고, 학습한 내용을 기반으로 어떠한 결정을 판단하거나 예측한다. 
딥러닝도 마찬가지로, 머신러닝에 포함되는 개념으로서, 인공신경망에서 발전한 형태이다. 머신러닝은 알고리즘이 부정확한 예측을 반환하면 엔지니어가 개입하여 조정해야 하지만, 딥러닝은 신경망 구조를 통해 예측 정확성 여부를 스스로 판단한다.

### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
#하이퍼파라미터 설정
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
#데이터 다운로드 및 불러오기. MNIST 손글씨 데이터
#train용 데이터
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() #딥러닝에 적합한 텐서형태로 변환
    ])
)
#test용 데이터
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() #딥러닝에 적합한 텐서형태로 변환
    ])
)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size) #train데이터 로드
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size) #test데이터 로드

examples = enumerate(train_set) #key, value를 enumerate 함수로 전달
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
#MNIST형태이므로 이미지 데이터 처리에 가장 적합한 딥러닝 형태인 CNN 구현
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #IN = 1 OUT = 10, Kernel size=5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) #과적합을 방지하기 위한 dropout
        self.mp = nn.MaxPool2d(2) # 풀링. 이미지의 특징을 솎아내기 위해, avg pooling 보단 maxpooling을 사용
        self.fc1 = nn.Linear(320,100) #fullyconnected layer 1. 선형 변환을 수행
        self.fc2 = nn.Linear(100,10) #fullyconnected layer 2. 선형 변환을 수행하여 최종적인 예측(10가지의 숫자 중 어느 것이 더 맞을까)


  def forward(self, x): #FeedForwardNeuralNetwork 구현
        x = F.relu(self.mp(self.conv1(x))) #input x를 통과 시키고 => 그 이후 conv 층을 통과시켜서(kernelsize=5*5) -> 이미지의 특성을 떼오기 위해 max풀링을 시키고 -> Activation함수 통과
        x = F.relu(self.mp(self.conv2(x))) #한번더
        x = self.drop2D(x) #과적합 방지 위해 dropout
        x = x.view(x.size(0), -1) #fc에 넣기 위해 차원변경squeeze
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x) #각 10개의 class 별 확률 계산해 주는 softmax 함수 통과.

In [ ]:

model = ConvNet().to(device) #gpu 가능하면 gpu 사용, 모델 정의
criterion = nn.CrossEntropyLoss().to(device)#loss 함수로서, classification task이므로 crossentropy loss 함수 차용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) #기존에 정의한 하이퍼파라미터대로, optimizer선정. (일반적으로 momentum과 rmsprop의 방식을 차용한 Adam이 좋다고 알려져있고 이를 사용)

In [ ]:
#훈련 start
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() #미분을 통해 얻은 기울기를 0으로 초기화하여, 새로운 루프를 돌 떄, 새로운 가중치에 대해 새로운 기울기를 구할 수 있음. 
        hypothesis = model(data) #Hypothesis
        cost = criterion(hypothesis, target) #prediction과 기존 target값을 비교하여 cost 계산
        cost.backward() #비용함수를 미분하여 gradient 계산
        optimizer.step() #가중치와 bias(W,b) 업데이트
        avg_cost += cost / len(train_loader) #평균적인 loss를 구함
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:

model.eval() #cnn model을 evalutation 모드로 변경-> Dropout 기능 비활성화
with torch.no_grad(): #평가 단계에서는 gradient를 계산하지 않으므로, 이 기능 비활성화시켜서 메모리 낭비x
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)#predictions
        preds = torch.max(out.data, 1)[1] 
        total += len(target) #전체 문제 수
        correct += (preds==target).sum().item()#몇개나 맞았느냐 
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.